In [1]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/
# https://stackoverflow.com/questions/25929319/how-to-iterate-over-pandas-multiindex-dataframe-using-index
# https://stackoverflow.com/questions/24495695/pandas-get-unique-multiindex-level-values-by-label
# https://stackoverflow.com/questions/55706391/pandas-crosstab-on-multiple-columns-then-groupby

# https://matplotlib.org/stable/gallery/pyplots/pyplot_text.html#sphx-glr-gallery-pyplots-pyplot-text-py

In [2]:
import pandas as pd
import numpy as np
from myUtils import pickle_load, pickle_dump

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

# # filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
# filename_symbols = path_data_dump + 'my_symbols.csv'  # symbols text file

# _filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV
filename_pickled_perf_rank_dict =  'perf_rank_dict'  # store symbols from performance rank results
filename_pickled_r_all_ranks =  'r_all_ranks'  # list of top 100 most common symbols from performance rank results
filename_pickled_df_a = 'df_OHLCV_clean'  # df adjusted OHLCV, dropped symbols with no vol and close
filename_pickled_df_c = 'df_close_clean'  # df close, dropped symbols with no vol and close

verbose = False  # True prints more output

#################
# look_back_days = -250 * 60  # subset df iloc days
look_back_days = -250 * 6  # subset df iloc days, 6 years of data
#################

In [3]:
print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)

Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close
Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV


In [4]:
# https://stackoverflow.com/questions/63826291/pandas-series-find-column-by-value
df = df_adjOHLCV[look_back_days::]
df_v = df.xs('Volume', level=1, axis=1)  # select only Volume columns
rows, cols = np.where(df_v == 0)  # row index, column index where trading volumes are zero
idx_no_volume = list(set(cols))
idx_no_volume.sort()
symbols_no_volume = df_v.columns[idx_no_volume]
print(f'symbols with no volume:\n{symbols_no_volume}')

symbols with no volume:
Index(['AAWW', 'AMCR', 'AVT', 'BCI', 'BHF', 'BKR', 'CCI', 'CHD', 'CNC', 'CNXC',
       'COUP', 'ESAB', 'FCFS', 'GEN', 'HUBB', 'IBKR', 'JJC', 'LHCG', 'PEAK',
       'PFG', 'SBNY', 'SIVB', 'UA', 'UMPQ', 'VC', 'VNT', 'WH', 'XEL'],
      dtype='object')


In [5]:
df_dif = df_v - df_v.shift(periods=1)
rows, cols = np.where(df_dif == 0)
idx_same_volume = list(set(cols))
idx_same_volume.sort()
idx_same_volume
symbols_same_volume = df_v.columns[idx_same_volume]
print(f'symbols with same volume:\n{symbols_same_volume}')

symbols with same volume:
Index(['AAWW', 'ACM', 'ACN', 'ADP', 'AGCO', 'AMCR', 'AMED', 'AMG', 'AMGN',
       'ANSS',
       ...
       'VAC', 'VICR', 'VRSK', 'VSAT', 'WERN', 'WEX', 'WTS', 'WTW', 'XEL',
       'YUM'],
      dtype='object', length=210)


In [6]:
df_c = df.xs('Close', level=1, axis=1)  # select only Close columns
df_c = df_c.fillna(0).copy()  # convert NaNs to zeros
rows, cols = np.where(df_c == 0)  # row index, column index where trading volumes are zero
idx_no_close = list(set(cols))
idx_no_close.sort()
symbols_no_close = df_c.columns[idx_no_close]
print(f'symbols with NaN close:\n{symbols_no_close}')

symbols with NaN close:
Index(['AAWW', 'ABNB', 'AIRC', 'ALGM', 'AM', 'ATCO', 'BHF', 'BJ', 'BRBR',
       'CAJ', 'CARR', 'CDAY', 'CEG', 'CHK', 'CHX', 'CNXC', 'COUP', 'CRWD',
       'CTVA', 'DDOG', 'DOW', 'DT', 'DTM', 'ESAB', 'ETH-USD', 'ETRN', 'FOX',
       'FOXA', 'FYBR', 'GEHC', 'GFS', 'GHC', 'GO', 'GXO', 'IAA', 'IR', 'JBGS',
       'JHG', 'JJC', 'KD', 'LCID', 'LHCG', 'MP', 'MRNA', 'NARI', 'NVST', 'NVT',
       'OGN', 'OTIS', 'PDD', 'PGNY', 'RCII', 'RFP', 'RIVN', 'SHC', 'SITM',
       'SIVB', 'SJR', 'SWAV', 'UMPQ', 'VICI', 'VNT', 'VSCO', 'WH', 'YETI',
       'ZM', 'ZS'],
      dtype='object')


In [7]:
symbols_drop = list(symbols_no_close) + list(symbols_no_volume) + list(symbols_same_volume) # combine symbols with no volume and no close
print(f'combined symbols with no volume, same volume and no close, inculdes duplicate symbols: {len(symbols_drop)}')
symbols_drop = list(set(symbols_drop))  # drop duplicate symbols
symbols_drop .sort()
df_a = df.drop(symbols_drop, axis=1, level=0)  # drop symbols from OHLCA df
df_c = df_close.iloc[look_back_days::]
df_c = df_c.drop(symbols_drop, axis=1)
print(f'unique symbols dropped from df_a (adjOLHLV) and df_c (Close): {len(symbols_drop)}')

combined symbols with no volume, same volume and no close, inculdes duplicate symbols: 305
unique symbols dropped from df_a (adjOLHLV) and df_c (Close): 276


In [8]:
print(f'symbols with no volume:   {len(symbols_no_volume):>5,}')
print(f'symbols with same volume: {len(symbols_same_volume):>5,}')
print(f'symbols with no close:    {len(symbols_no_close):>5,}\n')
print(f'symbols total before drop:                                        {len(df_close.columns):>5,}')
print(f'unique symbols dropped from df OHLCVA (df_a) and df Close (df_c): {len(symbols_drop):>5,}\n')
print('                                          symbols     rows')
print(f'df adjOHLCV (df_a) after dropped symbols:   {len(df_a.columns)/5:>5,.0f}    {len(df_a):>5,}')
print(f'df Close (df_c) after dropped symbols:      {len(df_c.columns):>5,}    {len(df_c):>5,}')


symbols with no volume:      28
symbols with same volume:   210
symbols with no close:       67

symbols total before drop:                                        1,593
unique symbols dropped from df OHLCVA (df_a) and df Close (df_c):   276

                                          symbols     rows
df adjOHLCV (df_a) after dropped symbols:   1,317    1,500
df Close (df_c) after dropped symbols:      1,317    1,500


In [9]:
pickle_dump(df_a, path_data_dump, filename_pickled_df_a)
print(f'pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_a}')
pickle_dump(df_c, path_data_dump, filename_pickled_df_c)
print(f'pickled df Close after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_c}')

pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCV_clean
pickled df Close after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_close_clean


In [10]:
from myUtils import list_dump

f_symbols_df_close_clean = 'symbols_df_close_clean.csv'  # symbols text file
symbols_df_c = list(df_c)  # column names in df_c
list_dump(symbols_df_c, path_data_dump, f_symbols_df_close_clean)# df_c.columns.to_csv(f_symbols_df_close_clean)

In [11]:
df_a.tail()

A                                                 \
                  Open        High         Low       Close     Volume   
Date                                                                    
2023-04-03  137.419998  138.089996  136.009995  137.860001  1551800.0   
2023-04-04  138.250000  139.110001  137.529999  138.210007  1778700.0   
2023-04-05  137.740005  139.270004  136.669998  138.089996  2312100.0   
2023-04-06  138.009995  138.130005  136.589996  138.029999  1206200.0   
2023-04-10  137.380005  137.380005  136.365005  136.529999   116282.0   

                   AA                                              ...  \
                 Open       High        Low      Close     Volume  ...   
Date                                                               ...   
2023-04-03  42.560001  42.950001  41.130001  41.770000  3272100.0  ...   
2023-04-04  41.439999  41.520000  39.310001  40.340000  4587400.0  ...   
2023-04-05  39.820000  40.189999  38.720001  39.430000  4251000.0  ...   
2023-04-06  39.119999  39.290001  38.080002  39.160000  5313600.0  ...   
2023-04-10  39.160000  40.150002  39.000000  39.939999   743068.0  ...   

                 ZUMZ                                                   ZWS  \
                 Open       High        Low      Close    Volume       Open   
Date                                                                          
2023-04-03  18.620001  18.980000  18.260000  18.950001  483800.0  21.480000   
2023-04-04  19.000000  19.129999  18.379999  18.510000  359400.0  21.490000   
2023-04-05  18.330000  18.340000  17.879999  18.309999  380000.0  20.360001   
2023-04-06  18.260000  18.530001  17.920000  18.459999  339000.0  20.309999   
2023-04-10  18.309999  18.790001  18.309999  18.740000   45875.0  20.030001   

                                                        
                 High        Low      Close     Volume  
Date                                                    
2023-04-03  21.580000  21.070000  21.440001   757500.0  
2023-04-04  21.500000  20.530001  20.610001  1194400.0  
2023-04-05  20.450001  19.950001  20.219999  1005700.0  
2023-04-06  20.379999  19.830000  20.090000  1543900.0  
2023-04-10  20.549999  20.100000  20.490000   134190.0  

[5 rows x 6585 columns]

In [12]:
df_c.tail()

,A,AA,AAL,AAP,AAPL,AB,ABB,ABBV,ABC,ABM,...,YY,ZBH,ZBRA,ZD,ZG,ZION,ZTO,ZTS,ZUMZ,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2023-04-03,137.860001,41.770000,14.420,122.690002,166.169998,35.700001,34.419998,160.419998,162.279999,44.712662,...,32.110001,128.039993,311.649994,77.800003,43.950001,29.120001,28.639999,167.660004,18.950001,21.440001
2023-04-04,138.210007,40.340000,14.160,121.339996,165.630005,34.810001,33.910000,159.779999,162.970001,43.270000,...,31.280001,127.919998,308.709991,76.510002,43.299999,27.709999,28.750000,167.070007,18.510000,20.610001
2023-04-05,138.089996,39.430000,13.890,120.209999,163.759995,34.119999,33.110001,160.800003,167.360001,43.349998,...,30.000000,128.589996,300.149994,75.809998,42.810001,28.629999,28.270000,167.649994,18.309999,20.219999
2023-04-06,138.029999,39.160000,14.010,120.690002,164.660004,34.110001,33.020000,161.550003,166.350006,43.840000,...,30.230000,129.169998,300.260010,76.699997,43.919998,29.389999,28.670000,168.940002,18.459999,20.090000
2023-04-10,136.529999,39.939999,14.065,120.935898,160.740097,34.320000,33.044998,160.350006,166.000000,44.080002,...,30.190001,127.675003,300.679993,77.160004,44.020000,29.750000,28.525000,167.500000,18.740000,20.490000
